## Model Registry

In [ ]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
print(registry.list_models())


model_info = registry.get_model_info("fai-rtdetr-l-obj365")

model_info.pprint()

## AutoDataset

In [ ]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.ports import DatasetLayout, DatasetSplitType, Task

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="aquarium", task=task, layout=layout, datasets_root_dir="../datasets")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

## Auto Model

In [ ]:
from focoos.auto_model import AutoModel

model = AutoModel.from_pretrained("fai-rtdetr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

# model = AutoModel.from_pretrained("experiments/aquarium2/model_info.json")

## Trainer

In [ ]:
from focoos.ports import TrainerArgs

# from focoos.trainer.trainer import FocoosTrainer

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)


model.train(args, train_dataset, valid_dataset)

In [ ]:
info = model.model_info
info.pprint()

## Validation only

In [ ]:
from focoos.ports import TrainerArgs

args = TrainerArgs(
    run_name="aquarium2",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=300,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.test(args, valid_dataset)

In [ ]:
import torch
from PIL import Image

from focoos.auto_model import AutoModel
from focoos.models.fai_rtdetr.processor import RTDetrProcessor

# post_processor = DetectionPostProcessor(valid_dataset.dataset.metadata)
model = AutoModel.from_pretrained("fai-rtdetr-m-coco")
post_processor = RTDetrProcessor()

data = Image.open("../image.jpg")

with torch.no_grad():
    out = model([data])
    out = post_processor.postprocess(out, data)

print(out)

## Image Classification

In [ ]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.ports import DatasetLayout, DatasetSplitType, Task

task = Task.CLASSIFICATION
layout = DatasetLayout.CLS_FOLDER
auto_dataset = AutoDataset(dataset_name="hymenoptera", task=task, layout=layout, datasets_root_dir="../datasets")
resolution = 224

train_augs, val_augs = get_default_by_task(task, resolution, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

In [ ]:
# Use the model for inference
from PIL import Image

from focoos.auto_model import AutoConfig, AutoModel
from focoos.nn.backbone.resnet import ResnetConfig
from focoos.ports import ModelFamily, ModelInfo, Task

# Create a configuration with a ResNet backbone
cls_config = AutoConfig.from_dict(
    ModelFamily.CLS,
    {
        "backbone_config": dict(ResnetConfig(model_type="resnet", depth=50, pretrained=True)),
        "num_classes": valid_dataset.dataset.metadata.num_classes,
        "resolution": resolution,
        "hidden_dim": 512,
        "dropout_rate": 0.2,
    },
)

model_info = ModelInfo(
    name="fai-cls-resnet50",
    description="ResNet50 model for classification",
    task=Task.CLASSIFICATION,
    classes=["cat", "dog", "bird"],
    im_size=224,
    model_family=ModelFamily.CLS,
    config=cls_config,
)
# Create the model
model = AutoModel.from_config(model_info)

image = Image.open("image.jpg")
outputs = model(image)

print(outputs)

In [ ]:
from focoos.ports import TrainerArgs

args = TrainerArgs(
    run_name="aquarium2",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=500,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)